# Pembuatan Dataset Menggunakan Spotify Web API
Laporan MBKM Bengkel Koding  
Rifky Ariya Pratama  
A11.2020.12628

In [3]:
# reference : https://www.youtube.com/watch?v=WAmEZBEeNmg

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import base64
from requests import post, get
import json
import csv
from sklearn import preprocessing

client_id = '97aeaf1e98f943edb1344ab86f71692a'
client_secret = '9f35e123caa7490b904ad6bcb98f4ba9'
playlistId = '1dtCMTYzAOzwKXqklxPJNS'

# 37i9dQZF1DXbrUpGvoi3TS - 1(similar sad songs)
# 1dtCMTYzAOzwKXqklxPJNS - 2(old songs, rock, rap)
# 0IN7IWKmIfwlEysGyWUuRg - 3(mix of modern electronic, pop, and rock)

dataset = []
dataset2 = []
dataset3 = []

def getToken():
    # gabungkan client_id dan client_secret
    auth_string = client_id + ':' + client_secret

    # encode ke base64
    auth_b64 = base64.b64encode(auth_string.encode('utf-8'))

    # url untuk mengambil token
    url = 'https://accounts.spotify.com/api/token'

    # header untuk mengambil token - sesuai dengan guide dari spotify
    headers = {
        'Authorization': 'Basic ' + auth_b64.decode('utf-8'),
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    # data untuk mengambil token - sesuai dengan guide dari spotify
    data = {'grant_type': 'client_credentials'}

    # kirim request POST ke spotify
    result = post(url, headers=headers, data=data)

    # parse response ke json
    json_result = json.loads(result.content)
    token = json_result['access_token']

    # ambil token untuk akses API
    return token

# panggil fungsi getToken()
getToken()

'BQABQX6qcjhpsln6-iGF4mRwFGjr5vDJ6s8SNY9P585NYCrNh3rhu4dL4HJ8muIyszbpOFMODAKyYpkUrmB55LfQDhlUZZri6Wpu3t75e7YMKQ6ZQ5I'

In [4]:
# pengambilan token untuk otorisasi API
def getAuthHeader(token):
    return {'Authorization': 'Bearer ' + token}

# pengambilan audio features dari track (lagu)
def getAudioFeatures(token, trackId):
    url = f'https://api.spotify.com/v1/audio-features/{trackId}' # endpoint untuk akses playlist
    headers = getAuthHeader(token) # ambil token untuk otorisasi, gunakan sebagai header
    result = get(url, headers=headers) # kirim request GET ke spotify
    json_result = json.loads(result.content) # parse response ke json

    # ambil data yang diperlukan dari response
    audio_features_temp = [ 
        json_result['danceability'],
        json_result['energy'],
        json_result['key'],
        json_result['loudness'],
        json_result['mode'],
        json_result['speechiness'],
        json_result['acousticness'],
        json_result['instrumentalness'],
        json_result['liveness'],
        json_result['valence'],
        json_result['tempo'],
    ]
    dataset2.append(audio_features_temp)

# pengambilan track (lagu) dari playlist
def getPlaylistItems(token, playlistId):
    url = f'https://api.spotify.com/v1/playlists/{playlistId}/tracks' # endpoint untuk akses playlist
    limit = '&limit=100' # batas maksimal track yang diambil
    market = '?market=ID' # negara yang tempat aplikasi diakses
    fields = '&fields=items%28track%28id%2Cname%2Cartists%2Cpopularity%2C+duration_ms%2C+album%28release_date%29%29%29' # format data dari track yang diambil
    url = url+market+fields+limit # gabungkan semua parameter
    headers = getAuthHeader(token) # ambil token untuk otorisasi, gunakan sebagai header
    result = get(url, headers=headers) # kirim request GET ke spotify
    json_result = json.loads(result.content) # parse response ke json
    # print(json_result)

    # ambil data yang diperlukan dari response
    for i in range(len(json_result['items'])):
        playlist_items_temp = []
        playlist_items_temp.append(json_result['items'][i]['track']['id'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['artists'][0]['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['popularity'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['duration_ms'])
        playlist_items_temp.append(
            int(json_result['items'][i]['track']['album']['release_date'][0:4]))
        dataset.append(playlist_items_temp)

    # ambil audio features dari semua track di dalam playlist
    for i in range(len(dataset)):
        getAudioFeatures(token, dataset[i][0])

    # gabungkan dataset dan dataset2
    for i in range(len(dataset)):
        dataset3.append(dataset[i]+dataset2[i])

    # convert dataset3 into csv
    with open('dataset.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "name", "artist", "popularity", "duration_ms", "year", "danceability", "energy", "key", "loudness", "mode",
                         "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])
        writer.writerows(dataset3)

    print('dataset.csv created')


token = getToken()
print('access token : '+token)

getPlaylistItems(token, playlistId)

# id, name, artist, popularity, duration_ms, year, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo

access token : BQDxuk_MOvO6ktskHNRRmtwM2vB9H5VUCrAGwm7NYgRpfZHjXkoJd1fP530wf0lOPc9zweU7f74MoMgW2FcN-dNQaHPiaKnpNCwYLsJeKmAHhrsjyAM
dataset.csv created


# Data Preprocessing

### Data Understanding

Berikut adalah dataset yang digunakan untuk analisis ini. Dataset ini berisi data lagu-lagu yang ada di Spotify. Dataset ini terdiri dari 19 fitur, yaitu:
1. `name` : Nama lagu
2. `artists` : Nama artis pembuat lagu
3. `year` : Tahun rilis lagu
4. `popularity` : Tingkat popularitas lagu. Nilai popularitas adalah antara 0 dan 100, dengan 100 menjadi nilai maksimal.
5. `key` : adalah kunci lagu dari lagu tersebut. kunci lagu direpresentasikan menggunakan angka menurut Pitch Class Notation Standard. Misalnya, 0 = C, 1 = C♯/D♭, 2 = D, dan seterusnya. Jika tidak ada kunci, maka nilai -1.
6. `mode` : Mode dari lagu tersebut. 1 = Major, 0 = Minor.
7. `explicit` : Lagu tersebut mengandung konten eksplisit atau tidak. 1 = Ya, 0 = Tidak.
8. `duration_ms` : Durasi lagu dalam milidetik.
9. `acousticness` : Tingkat intensitas akustik di dalam lagu tersebut. 0.0 = Lagu tersebut tidak akustik, 1.0 = Lagu tersebut sangat akustik.
10. `danceability` : Seberapa cocok lagu tersebut untuk mengiringi tarian. Nilai diambil berdasarkan kombinasi dari beberapa elemen lain seperti tempo, stabilitas ritme, kekuatan ritme, dan reguleritas umum. 0.0 = Lagu tersebut tidak cocok untuk mengiringi tarian, 1.0 = Lagu tersebut sangat cocok untuk mengiringi tarian.
11. `energy` : Energi adalah ukuran dari 0.0 hingga 1.0 dan mewakili ukuran perseptual dari intensitas dan aktivitas. Secara umum, lagu-lagu yang energik terasa cepat, keras, dan berisik. Sebagai contoh, death metal memiliki energi yang tinggi, sementara prelude Bach memiliki skor rendah dalam skala tersebut. Fitur perseptual yang berkontribusi pada atribut ini termasuk rentang dinamis, kebisingan terdengar, timbre, tingkat onset, dan entropi umum.
12. `instrumentalness` : Memprediksi apakah sebuah lagu tidak mengandung vokal. Suara "ooh" dan "aah" diperlakukan sebagai instrumen dalam konteks ini. Lagu rap atau spoken word secara jelas dianggap "vokal". Semakin mendekati nilai instrumentalness ke 1.0, semakin besar kemungkinan lagu tersebut tidak mengandung konten vokal. Nilai di atas 0.5 dimaksudkan untuk mewakili lagu instrumental, namun tingkat kepercayaan lebih tinggi saat nilai mendekati 1.0.
13. `loudness` : Kekuatan suara keseluruhan dari sebuah lagu dalam desibel (dB). Nilai kekuatan suara dihitung rata-rata sepanjang seluruh lagu dan bermanfaat untuk membandingkan kekuatan suara relatif antara lagu-lagu. Kekuatan suara adalah kualitas dari suara yang merupakan korelasi psikologis utama dari kekuatan fisik (amplitudo). Nilai-nilai biasanya berkisar antara -60 hingga 0 dB.
14. `liveness` : Mendeteksi keberadaan penonton dalam rekaman. Nilai liveness yang lebih tinggi mengindikasikan probabilitas yang lebih besar bahwa lagu tersebut dipentaskan secara langsung. Nilai di atas 0.8 memberikan kemungkinan kuat bahwa lagu tersebut dinyanyikan secara langsung.
15. `speechiness` : Speechiness mendeteksi keberadaan kata-kata yang diucapkan dalam sebuah lagu. Semakin mirip rekaman tersebut dengan ucapan (misalnya, acara talk show, buku audio, puisi), semakin mendekati nilai atribut 1.0. Nilai di atas 0.66 menggambarkan lagu yang kemungkinan terdiri sepenuhnya dari kata-kata yang diucapkan. Nilai antara 0.33 dan 0.66 menggambarkan lagu yang mungkin mengandung baik musik maupun ucapan, baik dalam bagian-bagian atau lapisan, termasuk dalam kasus seperti musik rap. Nilai di bawah 0.33 kemungkinan besar mewakili musik dan lagu-lagu yang tidak mirip dengan ucapan.
16. `tempo` : Rata rata kecepatan atau tempo dari sebuah lagu, diukur menggunakan beats (ketukan) per minute (BPM).
17. `valence` : Nilai antara 0-1 yang mendeskripsikan 'mood' dari sebuah lagu. Nilai yang kecil menandakan bahwa lagu mempunyai mood sedih, marah, depresi. Nilai yang besar menandakan lagu mempunyai mood yang senang, riang, euforia.

sumber : https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

### Load Data
Ketika cell diatas dijalankan, sebuah file csv bernama dataset.csv akan muncul. File ini berisi dataset yang akan kita olah. Muat dataset menggunakan dataframe dan check apakah dataset sudah benar.

In [5]:
# muat dataset
data = pd.read_csv('dataset.csv')
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,5R8J6WCnV1E2lDJgVoaA4P,b'Threats',b'Jean Grae',41,246293,2008,0.677,0.631,0,-6.363,1,0.0578,0.09860,0.000005,0.245,0.454,90.120
1,5ex7brQNeKhyh6wjQoMDr0,"b""Queen's Philosophy""",b'Sa-Roc',17,193750,2013,0.849,0.572,7,-10.119,1,0.3580,0.03350,0.000000,0.109,0.497,97.106
2,0t7k99JA9z0KaMw1BoEpk8,b'9th vs. Thought',b'Black Thought',39,167741,2018,0.751,0.933,7,-2.188,1,0.2940,0.02550,0.000000,0.071,0.458,93.032
3,1LM6EReMkAxuDXDF26ekl2,b'Get By',b'Talib Kweli',59,227426,2002,0.662,0.928,2,-4.688,1,0.3770,0.00324,0.000000,0.170,0.775,90.509
4,5DWbvNiVrH2u0Si9e21ZjW,b'We Comin',b'Sa-Roc',8,209655,2015,0.659,0.859,8,-2.767,1,0.4230,0.30100,0.000000,0.715,0.709,174.542


### Decode base64
Dataset harus di encode-decode agar semua karakter ter-standarisasi dan file bisa dibaca dan diproses, bebas dari korupsi data.  
Karena data yang diambil dari spotify web API di encode menggunakan bas64, maka kita akan men-decode data tersebut menggunakan algoritma yang sama.  
Namun ketika di decode, muncul efek samping seperti penambahan ('b) di depan data dan (') dibelakang data.  
Kita akan menghapus efek samping tersebut menggunakan teknik string slicing.

In [6]:
# Hapus karakter yang tidak perlu pada kolom artist dan name
data['artist'] = data['artist'].map(lambda x: str(x)[2:-1])
data['name'] = data['name'].map(lambda x: str(x)[2:-1])

data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,5R8J6WCnV1E2lDJgVoaA4P,Threats,Jean Grae,41,246293,2008,0.677,0.631,0,-6.363,1,0.0578,0.09860,0.000005,0.245,0.454,90.120
1,5ex7brQNeKhyh6wjQoMDr0,Queen's Philosophy,Sa-Roc,17,193750,2013,0.849,0.572,7,-10.119,1,0.3580,0.03350,0.000000,0.109,0.497,97.106
2,0t7k99JA9z0KaMw1BoEpk8,9th vs. Thought,Black Thought,39,167741,2018,0.751,0.933,7,-2.188,1,0.2940,0.02550,0.000000,0.071,0.458,93.032
3,1LM6EReMkAxuDXDF26ekl2,Get By,Talib Kweli,59,227426,2002,0.662,0.928,2,-4.688,1,0.3770,0.00324,0.000000,0.170,0.775,90.509
4,5DWbvNiVrH2u0Si9e21ZjW,We Comin,Sa-Roc,8,209655,2015,0.659,0.859,8,-2.767,1,0.4230,0.30100,0.000000,0.715,0.709,174.542


### Cek Nilai Kosong
Sebagai efek samping dari encode-decode, terkadang kolom nama kosong. Maka kita harus menemukan data tersebut dan menghapusnya, agar algoritma dapat memproses data yang benar.  

In [7]:
#delete empty string in name column
data = data[data['name'] != '']

#reset index
data = data.reset_index(drop=True)

data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,5R8J6WCnV1E2lDJgVoaA4P,Threats,Jean Grae,41,246293,2008,0.677,0.631,0,-6.363,1,0.0578,0.09860,0.000005,0.245,0.454,90.120
1,5ex7brQNeKhyh6wjQoMDr0,Queen's Philosophy,Sa-Roc,17,193750,2013,0.849,0.572,7,-10.119,1,0.3580,0.03350,0.000000,0.109,0.497,97.106
2,0t7k99JA9z0KaMw1BoEpk8,9th vs. Thought,Black Thought,39,167741,2018,0.751,0.933,7,-2.188,1,0.2940,0.02550,0.000000,0.071,0.458,93.032
3,1LM6EReMkAxuDXDF26ekl2,Get By,Talib Kweli,59,227426,2002,0.662,0.928,2,-4.688,1,0.3770,0.00324,0.000000,0.170,0.775,90.509
4,5DWbvNiVrH2u0Si9e21ZjW,We Comin,Sa-Roc,8,209655,2015,0.659,0.859,8,-2.767,1,0.4230,0.30100,0.000000,0.715,0.709,174.542


### Feature Selection
Tujuan utama kita adalah untuk melakukan clustering terhadap karakteristik lagu. Dalam dataset, ada beberapa fitur yang tidak mencerminkan karakteristik dari sebuah lagu, maka kita harus menghapus fitur-fitur tersebut.  
anda bisa melihat deskripsi dari fitur tersebut di section data understanding diatas.

In [8]:
# drop name artist and year column
data2 = data.copy()
data2 = data2.drop(['artist', 'name', 'year', 'popularity', 'key','duration_ms', 'mode', 'id'], axis=1)

data2.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.677,0.631,-6.363,0.0578,0.09860,0.000005,0.245,0.454,90.120
1,0.849,0.572,-10.119,0.3580,0.03350,0.000000,0.109,0.497,97.106
2,0.751,0.933,-2.188,0.2940,0.02550,0.000000,0.071,0.458,93.032
3,0.662,0.928,-4.688,0.3770,0.00324,0.000000,0.170,0.775,90.509
4,0.659,0.859,-2.767,0.4230,0.30100,0.000000,0.715,0.709,174.542


### Normalisasi MinMax
beberapa fitur mempunyai batasan nilai yang berbeda-beda dan jaraknya cukup jauh. sebagai contoh, di fitur loudness, batasan nilai adalah -60 sampai -3, sedangkan di variabel instrumentalness, batasan nilai adalah 0 sampai 3.490000e-01. Untuk data min max selengkapnya bisa dicek di code dibawah.

nilai spread yang cukup besar ini dapat mempengaruhi hasil akhir clusterisasi, maka kita harus melakukan proses standarisasi agar semua fitur mempunyai batasan yang sama, yaitu 0 sampai 1

In [9]:
# check min max masing masing variable
data2.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000
mean,0.697490,0.710490,-6.662950,0.201792,0.198313,0.072261,0.226766,0.57168,106.059930
std,0.145967,0.148948,2.480507,0.129588,0.214676,0.205381,0.180887,0.19745,30.011474
min,0.245000,0.357000,-15.094000,0.029000,0.000087,0.000000,0.036000,0.14400,76.206000
25%,0.631750,0.596250,-7.756000,0.074700,0.030675,0.000000,0.100675,0.45050,89.746000
50%,0.704500,0.712500,-6.599500,0.196500,0.114500,0.000003,0.143000,0.56450,93.998000
75%,0.799500,0.837250,-5.105500,0.297000,0.320250,0.003790,0.298000,0.70250,103.659500
max,0.942000,0.961000,-1.518000,0.510000,0.863000,0.915000,0.855000,0.95900,192.296000


In [10]:
from sklearn import preprocessing

# normalize all data to 0 and 1
x = data2.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data2 = pd.DataFrame(x_scaled)

# convert to dataframe
data2.columns = ['acousticness','danceability','energy','instrumentalness','loudness', 'liveness', 'speechiness', 'tempo','valence']

# data2.head()
data2.describe()

,acousticness,danceability,energy,instrumentalness,loudness,liveness,speechiness,tempo,valence
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.649197,0.585248,0.621026,0.359235,0.229716,0.078974,0.232926,0.524761,0.257162
std,0.209422,0.246602,0.182713,0.269414,0.248781,0.224460,0.220864,0.242270,0.258519
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.554878,0.396109,0.540513,0.095010,0.035447,0.000000,0.078968,0.376074,0.116634
50%,0.659254,0.588576,0.625700,0.348233,0.132589,0.000003,0.130647,0.515951,0.153260
75%,0.795552,0.795116,0.735747,0.557173,0.371026,0.004142,0.319902,0.685276,0.236485
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Dimensionality Reduction using Principal Component Analysis (PCA)

Setelah kita menghapus fitur yang tidak terpakai, kita masih mempunyai 9 fitur. Kita tidak dapat melakukan proses clustering dengan 9 fitur, karena proses clustering hanya bisa dilakukan dengan maksimal 2 fitur. Oleh karena itu diperlukan metode dimensionality reduction dengan Principal Component Analysis untuk mengkondensasi 9 fitur menjadi 2 fitur. Nilai intrinsik yang ada di 9 fitur tidak akan hilang, nilai tersebut akan direpresentasikan ke bentuk yang lebih kecil, menjadi 2 fitur. 

Sebagai sebuah analogi, seorang fotografer yang sedang berwisata ke sebuah taman margasatwa di dekat pegunungan. Fotografer melihat banyaknya keindahan alam disana. Pemikiran pertama yang muncul adalah untuk memotret semua keindahan alam yang ada, fotografer tidak peduli akan mengambil 100, 500 bahkan 1000 foto sekalipun. Namun tiba tiba sang fotografer teringat bahwa dia hanya mempunyai memori SD card 4gb, jadi dia tidak bisa mengambil foto sebayak itu. Pengambilan foto secara keseluruhan ini bisa dianaologikan sebagai dataset awal kita yang berukuran besar dan tidak bisa diolah.

Lalu apa solusinya? Fotografer menganalisa keindahan-keindahan alam disekitarnya dan menemukan bahwa beberapa bagian tidak perlu difoto. Seperti kotoran-kotoran hewan yang ada di lapangan, pohon tumbang yang berisi belatung, daun-daun yang dimakan parasit. Hal ini bisa dianalogikan sebagai fitur yang tidak mengandung makna khusus, dan bisa dihapus atau dikurangi.

Mengetahui hal ini, sang fotografer mulai mencari keindahan alam sebenarnya yang patut diabadikan, seperti gunung yang membelah awan, gradasi warna  langit yang indah, dan bunga-bunga yang bermekaran. Fotografer memotret keindahan tersebut dan hanya menyimpan 5 foto. Hal ini dapat dianalogikan bahwa beberapa fitur mempunyai banyak makna atau value, dan fitur tersebut patut disimpan atau difokuskan. 

Dataset awalnya mempunyai ukuran besar dan fitur yang banyak. Menggunakan PCA, dataset di proses, dihilangkan fitur yang tidak bermakna, dan memfokuskan fitur yang bermakna. Ukuran dataset berkurang, namun value atau makna dataset secara keseluruhan masih sama seperti posisi awal.

In [11]:
# Lakukan PCA untuk mengurangi jumlah fitur menjadi 2 fitur saja
pca = PCA(n_components=2)
pca.fit(data2)
pca_data = pca.transform(data2)

# buat dataframe dari hasil pca
pca_df = pd.DataFrame(data=pca_data, columns=['x', 'y'])
pca_df.head()

# plot pca_df using plotly
fig = px.scatter(pca_df, x='x', y='y', title='PCA')
fig.show()

# Clustering
Clustering adalah proses pengelompokan data atau objek-objek serupa menjadi kelompok-kelompok yang lebih homogen berdasarkan kesamaan fitur atau karakteristik tertentu.

## K-Means
Algoritma k-means adalah metode clustering yang mengelompokkan data menjadi beberapa kelompok berdasarkan jaraknya ke pusat kelompok yang ditentukan secara iteratif.

Berikut ini penjelasan langkah-langkah algoritma clustering k-means:

1. Inisialisasi: Secara acak, pilih K titik data awal sebagai pusat-pusat awal (centroid). Pusat-pusat ini mewakili titik-titik pusat kelompok awal.

2. Penugasan: Untuk setiap titik data dalam himpunan data, hitung jarak antara titik tersebut dengan setiap pusat. Berikan titik data ke kelompok yang memiliki pusat terdekat dengan titik tersebut. Langkah ini biasanya menggunakan jarak Euclidean, tetapi bisa juga menggunakan metrik jarak lainnya.

3. Pembaruan: Setelah mengalokasikan semua titik data ke dalam kelompok-kelompok, hitung kembali pusat kelompok (centroid) dari setiap kelompok. Pusat kelompok dihitung sebagai nilai rata-rata dari semua titik data yang termasuk dalam kelompok tersebut.

4. Iterasi: Ulangi langkah penugasan dan pembaruan secara berulang hingga konvergensi. Konvergensi terjadi ketika penugasan titik-titik data ke kelompok-kelompok tidak berubah lagi atau ketika mencapai batas iterasi maksimum yang telah ditentukan sebelumnya.

5. Hasil Akhir: Setelah algoritma konvergen, titik-titik data dikelompokkan ke dalam K kelompok yang berbeda. Setiap titik data termasuk dalam kelompok yang memiliki pusat terdekat dengan titik tersebut.



In [12]:
# rubah bentuk data ke list 
data2 = list(zip(pca_df['x'], pca_df['y']))

# fit kmeans model
kmeans = KMeans(n_init=10, max_iter=1000).fit(data2)

# make scatter plot using plotly
fig = px.scatter(pca_df, x='x', y='y', color=kmeans.labels_, color_continuous_scale='rainbow', hover_data=[data.artist, data.name])
fig.show()

Kita menggunakan library sklearn dengan function sklearn.cluster.KMeans
Function ini mempunyai beberapa parameter, antara lain 
- **n_clusters**, jumlah cluster yang dipilih. Jika tidak diisi maka cluster akan di generate secara otomatis, tergantung fitur datanya.
- **n_init or init**, berapa kali algoritma akan dijalankan menggunakan awalan (seed) centroid yang berbeda. nilai default dari n_init adalah 10, sedangkan untuk init adalah 1
- **max_iter**, jumlah iterasi dari algoritma k-means yang dijalankan setiap run
- **tol**, singkatan dari *tolerance*, adalah toleransi relatif untuk menyatakan konvergensi. nilai lebih besar berarti lebih banyak konvergensi.
- **verbose**, jika true code akan menghasilkan statistik detil
- **random_state**, Menentukan pembangkitan angka acak untuk inisialisasi pusat klaster. Gunakan bilangan bulat (int) untuk membuat randomness menjadi deterministik.
- **copy_x**, jika false, data asli akan diubah sedikit untuk mempercepat proses training. Jika true maka data tidak diubah.
- **algorithm**, menentukan algoritma k-means yang akan digunakan. default adalah 'lloyd'. Variasi 'elkan' lebih efisien untuk data clustering yang nilainya sangat jelas.

Parameter yang kita gunakan adalah **n_init=10** untuk menambah konsistensi hasil dan **max_iter=1000** untuk menambah akurasi clustering. 

# Kesimpulan

Untuk project machine learning clustering ini, kita menggunakan dataset lagu yang diambil dari spotify. 

Untuk melakukan proses pengambilan lagu, kita membutuhkan akses API dari spotify. Pertama kita harus mendaftar sebagai spotify developer di web resmi spotify. Lalu kita mengambil sebuah token menggunakan kredensial yang kita dapat dari akun developer kita. Sebuah token memungkinkan kita untuk mengambil data dari spotify web API secara langsung. user menginput url playlist dan aplikasi akan mengambil semua lagu didalam playlist tersebut dan membuat dataset secara otomatis. 

Dataset yang dihasilkan sudah cukup solid karena spotify web API memang dikenal mempunyai kualitas tinggi. Namun kita tetap melakukan pengecekan konsistensi data. Ketika data diambil dari web, beberapa lagu/artist mempunyai karakter yang tidak dapat ditampilkan seperti huruf kanji, maka kita harus encode-decode karakter tersebut ke standar utf-8. Namun terkadang proses dari encode-decode menghasilkan sebuah string kosong. Walaupun fitur yang lain tidak kosong, data ini tidak bisa kita identifikasi, jadi kita akan hapus data ini.

Setelah di preproses, kita akan melakukan proses dimensionality reduction menggunakan principal component analysis (PCA). Proses clustering menggunakan scatter plot mengharuskan data mempunyai 2 fitur. Tujuan dari proses PCA adalah mengurangi jumlah fitur dari 9 menjadi 2 tanpa mengurangi makna yang ada didalam dataset. 

Kita melakukan clustering dataset menggunakan algoritma K-Means dengan titik cluster yang diatur secara otomatis. Adapun parameter yang kita masukkan ke algoritma yaitu **n_init=10** untuk menambah konsistensi hasil dan **max_iter=1000** untuk menambah akurasi clustering. 

Hasilnya dataset terbagi menjadi 7 buah cluster (secara otomatis), dan masing masing lagu di dalam cluster mempunyai keunikan tersendiri. Keunikan tersebut dapat direpresentasikan dengan tempo lagu, jenis melodi, jenis instrumen, overall 'vibe' dari lagu, dan lain lain. Jumlah cluster dapat diatur menggunakan parameter **n_cluster**. 

# Deploy ke Streamlit
Streamlit adalah framework open-source untuk mengembangkan aplikasi web interaktif dengan menggunakan bahasa pemrograman Python.
Tujuannya adalah menyederhanakan proses pembuatan aplikasi web dengan memungkinkan pengembang untuk membuat aplikasi dengan mudah menggunakan kode Python yang sederhana dan familiar.

## Buat file bernama streamlit_app.py
Langkah pertama adalah membuat sebuah file baru bernama streamlit_app.py. Kita akan masukkan semua koding yang telah kita buat sebelumnya ke dalam satu file ini.  
Kita telah membahas proses pengambilan dataset, dan telah dijelaskan bahwa kita harus membuat beberapa fungsi seperti getToken(), getPlaylistItems().  Masukkan semua fungsi-fungsi tersebut ke dalam file streamlit_app.py.  
Selain itu, pada bab data preprocessing, terdapat banyak potongan code untuk pemrosesan data. Masukkan semua potongan code tersebut ke dalam sebuah function bernama dataProcessing().  
Akhirnya, berikut fungsi-fungsi yang ada di dalam streamlit_app.py:  
- **getToken()**, untuk mengambil token
- **getAuthHeader()**, untuk mengisi header request dengan token yang didapat
- **getAudioFeatures()**, untuk mendapatkan data karakteristik lagu
- **getPlaylistItems()**, untuk mengambil lagu-lagu yang ada di dalam playlist
- **dataProcessing()**, untuk memproses dataset dan menghasilkan clustering

## Upload ke github repository
Jika semua coding sudah selesai, maka kita harus meng-upload semua coding kita ke github. 
- login ke github
- di dashboard github, klik `New` untuk membuat repository baru
- beri nama repositori, deskripsi, dan **pastikan bahwa repository bersifat public**. Lalu klik create repository
- kembali ke code editor, masukkan file streamlit_app.py ke dalam folder
- ketik `git init`
- ketik `git add .`
- ketik `git remote add origin https://github.com/[username]/[repository_name].git` contohnya g

## Buat Akun Streamlit
- Pergi ke streamlit.io
- Buat akun/masuk menggunakan akun
- Sangat direkomendasikan masuk menggunakan akun github

## Create new app